In [ ]:
import pyfar as pf
import numpy as np
from matplotlib import pyplot as plt


In [ ]:
def plot_time(data, sampling_rate, log=True, log_prefix=20, *args, **kwargs):
    n_samples = data.shape[-1]
    times = np.arange(n_samples)/sampling_rate
    if log:
        data_db = log_prefix*np.log10(np.abs(data))
        plt.plot(times, data_db.T, *args, **kwargs)
        max_db = np.nanmax(data_db)
        ax = plt.gca()
        ax.set_ylim((max_db-80+5, max_db+5))
        ax.set_ylabel('Amplitude [dB re 1]')
    else:
        plt.plot(times, data.T, *args, **kwargs)
        ax.set_ylabel('Amplitude [re 1]')
        
    ax.set_xlabel('Time [s]')
    plt.grid(True)
    plt.legend()

# Room Acoustics Toolbox in Python
The Room Acoustics Toolbox in Python is a collection of functions for commonly used calculcations in room acoustics.
## Getting Started
In this notebook, the most relevant functions of the toolbox are outlined.
### 1 Generate an Analytic Room Impulse Response
In the following it is outlined, how the transfer function of a rectangular room based on the analytic model [1] can be generated using the toolbox.

In [ ]:
from pyrato import analytic

In order to generate an analytic transfer function of a room, the fundamental characteristics of the scenario have to be spectified. The RIR is generated by the function `rectangular_room_rigid_walls(dimensions=..., source=..., receiver=..., reverberation_time=..., max_freq=..., samplingrate=..., speed_of_sound=..., n_samples=...)`, which is part of the `analytic` module.

In [ ]:
sampling_rate = 44100
room_dimensions = [3, 2, 2]
src_pos = [0.33, 1.55, 1]
rec_pos = [2.21, 0.34, 1.2]
t_60 = 2
max_freq = 1000
n_samples = 2**18
speed_of_sound = 343.9

In [ ]:
rir_1 = analytic.rectangular_room_rigid_walls(
    dimensions=room_dimensions, source=src_pos, receiver=rec_pos, reverberation_time=t_60, max_freq=max_freq,
    samplingrate=sampling_rate, speed_of_sound=speed_of_sound, n_samples=n_samples)[0]

rir_1.time /= np.amax(np.abs(rir_1.time)) # Normalize 

# Plot the result:
plt.figure(figsize=(10, 4))
pf.plot.time(rir_1, dB=True)

### 2 Introducing Noise
In order to demonstrate the provided energy decay curve noise handling methods, it is necessary to introduce some Gaussian noise on the channel.

In [ ]:
psnr = 50
awgn = pf.signals.noise(rir_1.n_samples, rms=10**(-(psnr) / 20))
rir_1_noise = rir_1.copy() + awgn

plt.figure(figsize=(10, 4))
pf.plot.time(rir_1_noise, label='RIR + Noise', dB=True)

### 3 Estimate the Noise Energy of a RIR
The `edc_noise_handling` module provides various methods for noise compensation of room impulse responses. 

In [ ]:
import pyrato as ra
import pyfar as pf

The interval can be chosen as percentages of the RIR. The estimation interval `interval=[0.9, 1.0]` corresponds to the last 10% of the RIR.

In [ ]:
interval = [0.9, 1.]

In [ ]:
plt.figure(figsize=(10, 4))
pf.plot.time(rir_1_noise, dB=True, label='RIR')
plt.axvline(rir_1_noise.signal_length*interval[0], color='red', label='Interval')
plt.axvline(rir_1_noise.signal_length*interval[1], color='red')

noise_energy = ra.dsp.estimate_noise_energy(
    rir_1_noise, interval=interval, is_energy=False)

noise_energy_db = 10*np.log10(noise_energy)
plt.axhline(noise_energy_db, color='k', linestyle='--', label='Noise Level')

plt.legend(loc='upper center', ncol=3)

output_string = "The estimated noise level is at {} dB.".format(noise_energy_db)
print(output_string)

### 4 EDC Noise Compensation

The functions regarding EDC noise compensation always accept the following parameters. The examples below always utilize the default parameters, e.g. a time shift is performed.
```
Parameters
----------
data : ndarray, double
    The room impulse response with dimension [..., n_samples]
sampling_rate: integer
    The sampling rate of the room impulse response.
freq: integer OR string
    The frequency band. If set to 'broadband',
    the time window of the Lundeby-algorithm will not be set in dependence
    of frequency.
noise_level: ndarray, double OR string
    If not specified, the noise level is calculated based on the last 10
    percent of the RIR. Otherwise specify manually for each channel
    as array.
is_energy: boolean
    Defines, if the data is already squared.
time_shift : boolean
    Defines, if the silence at beginning of the RIR should be removed.
channel_independent : boolean
    Defines, if the time shift and normalizsation is done
    channel-independently or not.
normalize : boolean
    Defines, if the energy decay curve should be normalized in the end
    or not.
plot: Boolean
    Specifies, whether the results should be visualized or not.

Returns
-------
energy_decay_curve: ndarray, double
    Returns the noise handeled edc.
```
#### 4.1 Full Impulse Response
When considering the full room impulse response for the EDC calculation, the plot below shows that the noise in the RIR introduces an error in the estimated EDC.

$$ E_{\mathrm{Full}}(t) = \int_t^{t_{IR}} h^2(\tau) d\tau  $$

$E(t)$ denotes the energy decay curve, $t$ the time, $t_{IR}$ the length of the impulse response and $h(\tau)$ the impulse response.



In [ ]:
start_sample = pf.dsp.find_impulse_response_start(rir_1_noise)
rir_noise_shifted = pf.dsp.time_shift(rir_1_noise, -start_sample, mode='linear')
edc_full_rir = ra.edc.schroeder_integration(rir_noise_shifted, is_energy=False)

start_sample = pf.dsp.find_impulse_response_start(rir_1)
rir_shifted = pf.dsp.time_shift(rir_1, -start_sample, mode='linear')
edc_full_rir_compare = ra.edc.schroeder_integration(rir_shifted, is_energy=False)
edc_full_rir_compare.time /= np.atleast_2d(edc_full_rir_compare.time[..., 0]).T
edc_full_rir.time /= np.atleast_2d(edc_full_rir.time[..., 0]).T

plt.figure(figsize=(10, 4))
pf.plot.time(edc_full_rir_compare, dB=True, log_prefix=10, label='Reference')
pf.plot.time(edc_full_rir, dB=True, log_prefix=10, label='With Noise')
plt.legend()
plt.ylim(-60)

In [ ]:
from pyrato import dsp

#### 4.2 Truncation

The RIR can be truncated to the intersection time $t_i$, where the decaying impulse response intersects with the estimated noise floor. The estimation of the intersection time after Lundeby et al. [2] is visualized below.

$$ E_\mathrm{Trunc}(t) = \int_t^{t_{i}} h^2(\tau) d\tau  $$

In [ ]:
intersection_time, late_reveberation_time, noise_level_lundeby = \
    ra.edc.intersection_time_lundeby(
        rir_1_noise, freq='broadband', is_energy=False,
        time_shift=True, channel_independent=False, plot=True)

output_string = "The estimated intersection time is {}s, the late reverberation time is {}s and the estimated noise is {}dB.".format(intersection_time, late_reveberation_time, 10*np.log10(np.abs(noise_level_lundeby)))
print(output_string)

The next plot shows, how the noise error reduces significantly if the truncated RIR is utilized.

In [ ]:
edc_truncation = ra.edc.energy_decay_curve_truncation(
        rir_1_noise, freq='broadband', is_energy=False, time_shift=True,
        channel_independent=False, normalize=True, plot=False)

plt.figure(figsize=(10, 4))
pf.plot.time(edc_full_rir_compare, dB=True, log_prefix=10, label='Reference')
pf.plot.time(edc_truncation, dB=True, log_prefix=10, label='Truncated')
plt.legend()
plt.ylim(-60)
plt.xlim(0, 3)

#### 4.3 Chu's Method

Chu [3] proposed a noise compensation method, where the noise energy is subtracted from the RIR in advance. This yields a even more noise compensated EDC.

$$ E(t)_\mathrm{Chu} = \int_t^{t_{IR}} (h^2(\tau) - N_{est}^2) d\tau  $$

In [ ]:
edc_chu_auto = ra.edc.energy_decay_curve_chu(
        rir_1_noise, noise_level='auto', is_energy=False, time_shift=True,
        channel_independent=False, normalize=True, plot=False)

In [ ]:
noise_level_man = ra.dsp.estimate_noise_energy(rir_1_noise, interval=[0.6, 1.0], is_energy=False)
edc_chu_man = ra.edc.energy_decay_curve_chu(
        rir_1_noise, noise_level=noise_level_man, is_energy=False, time_shift=True,
        channel_independent=False, normalize=True, plot=False)

In [ ]:
edc_chu_noise_lundeby = ra.edc.energy_decay_curve_chu(
        rir_1_noise, noise_level=noise_level_lundeby, is_energy=False, time_shift=True,
        channel_independent=False, normalize=True, plot=False)

In [ ]:
plt.figure(figsize=(10, 4))
pf.plot.time(edc_full_rir_compare, dB=True, log_prefix=10, label='Reference')
pf.plot.time(edc_chu_auto, dB=True, log_prefix=10, label='Subtraction, Last 10% Estimation')
pf.plot.time(edc_chu_man, dB=True, log_prefix=10, label='Subtraction, Manual Noise Estimation')
pf.plot.time(edc_chu_noise_lundeby, dB=True, log_prefix=10, label='Subtraction, Lundeby Noise Estimation')
plt.legend()
plt.ylim(-60)
plt.xlim(0, 3);

#### 4.4 Lundeby's Method

Lundeby et al. [2] proposed a noise compensation method, where the RIR is truncated to the intersection time and additionally intruced a correction term $C_\mathrm{Comp}$.

$$ E_\mathrm{Lundeby}(t) = \int_t^{t_i} h^2(\tau) d\tau + C_\mathrm{Comp}  $$

$$C_\mathrm{Comp} = p_{i}^2 \cdot T_{60, late} \cdot \frac{1}{6 \cdot ln(10)} \cdot R_s $$

$p_{i}^2$ denotes the squared sound pressure at the intersection time, $T_{60, late}$ the late reverberation time after Lundeby and $R_s$ the sampling rate.

TO-DO: Reference for correction term

In [ ]:
edc_lundeby = ra.edc.energy_decay_curve_lundeby(
        rir_1_noise, is_energy=False, time_shift=True,
        channel_independent=False, normalize=True, plot=False)

plt.figure(figsize=(10, 4))
pf.plot.time(edc_full_rir_compare, dB=True, log_prefix=10, label='Without noise')
pf.plot.time(edc_lundeby, dB=True, log_prefix=10, label='Lundeby')
plt.legend()
plt.ylim(-60)
plt.xlim(0,3)

#### 4.5 Lundeby and Chu Combined

When combining all methods, the influence of the noise can be minimized [4].

$$ E_\mathrm{Chu, Lundeby}(t) = \int_t^{t_i} (h^2(\tau) - N_{est}^2) d\tau + C_\mathrm{Comp}  $$

In [ ]:
edc_chu_lundeby = ra.edc.energy_decay_curve_chu_lundeby(
        rir_1_noise, freq='broadband', is_energy=False, time_shift=True,
        channel_independent=False, normalize=True, plot=False)

plt.figure(figsize=(10, 4))
pf.plot.time(edc_full_rir_compare, dB=True, log_prefix=10, label='Reference')
pf.plot.time(edc_chu_lundeby, dB=True, log_prefix=10, label='Chu/Lundeby')
plt.legend()
plt.ylim(-60)
plt.xlim(0,3)

#### 4.6 Comparison of All Noise Compensation Methods

In [ ]:
plt.figure(figsize=(10, 4))
pf.plot.time(edc_full_rir_compare, dB=True, log_prefix=10, label='Reference')
pf.plot.time(edc_full_rir, dB=True, log_prefix=10, label='No Compensation')
pf.plot.time(edc_truncation, dB=True, log_prefix=10, label='Truncation')
pf.plot.time(edc_chu_noise_lundeby, dB=True, log_prefix=10, label='Chu')
pf.plot.time(edc_lundeby, dB=True, log_prefix=10, label='Lundeby')
pf.plot.time(edc_chu_lundeby, dB=True, log_prefix=10, label='Chu/Lundeby')
plt.xlabel('Time [s]')
plt.ylabel('EDC [dB]')
plt.legend()
plt.ylim(-60)
plt.xlim(0,3)

In [ ]:
Tx = 'T30'

edcs = [edc_full_rir_compare, edc_truncation, edc_chu_auto, edc_lundeby, edc_chu_lundeby]
reverberation_times = np.zeros(len(edcs))
for idx, edc in enumerate(edcs):
    reverberation_times[idx] = \
        ra.parameters.reverberation_time_linear_regression(
            edc, T=Tx)

In [ ]:
plt.figure(figsize=(10, 4))
plt.bar(np.arange(len(reverberation_times)), reverberation_times)
plt.grid(True)
ax = plt.gca()
ax.set_xticklabels(['', 'Reference', 'Truncation', 'Chu', 'Lundeby', 'Chu/Lundeby']);
ax.set_ylabel(Tx + ' [s]')

## References
[1] H. Kuttruff, Room acoustics, pp. 64-66, 4th Ed. Taylor & Francis, 2009.

[2] Lundeby, Virgran, Bietz and Vorlaender - Uncertainties of Measurements in Room Acoustics - ACUSTICA Vol. 81 (1995)

[3] W. T. Chu. “Comparison of reverberation measurements using Schroeder’s impulse method and decay-curve averaging method”. In: Journal of the Acoustical Society of America 63.5 (1978), pp. 1444–1450.

[4] M. Guski, “Influences of external error sources on measurements of room acoustic parameters,” 2015.
